# Remind user on equipment to bring

In [1]:
# import libraries
import datetime
import pandas as pd
import time
import os
import sys

In [2]:
# open subscriber list
#project_home = os.environ['project_home'] # <- actual run
project_home = '/home/kuanpern/airflow_analysis_pipe'
datadir = project_home + '/data/'
filename = '{datadir}/subscribers.list'.format(datadir=datadir)
print 'filename:', filename

filename: /home/kuanpern/airflow_analysis_pipe/data//subscribers.list


In [3]:
df_subscribers = pd.read_csv(filename, sep=',')
subscribers = df_subscribers.transpose().to_dict().values()
subscribers

[{'email': 'admin@somecompany.com', 'username': 'David'},
 {'email': 'user004@gmail.com', 'username': 'kuanpern'}]

In [4]:
# read the "pipe" file
filename = '{datadir}/equipment_alert.csv'.format(datadir=datadir)
df = pd.read_csv(filename, sep='\t')
df

timestamp     equipments
0  2018-04-15T16:10:52.388614  umbrella,coat

In [5]:
# extract today
_today = datetime.datetime.now().isoformat().split('T')[0]
dets = [_.split('T')[0] == _today for _ in df['timestamp']]
df_today = df[dets]
df_today

timestamp     equipments
0  2018-04-15T16:10:52.388614  umbrella,coat

In [6]:
# find latest record
rec = df_today.sort_values(by='timestamp').iloc[-1].to_dict()
content = ', '.join(rec['equipments'].split(','))

### send email

In [7]:
# send email function
def utils_sendMail(emailUser, recipient, subject, text, attachFile = None):
    import subprocess
    import tempfile
    fp = tempfile.NamedTemporaryFile()
    fp.write(text)
    fp.seek(0)

    cmd_template = '''mail -aFrom:{emailUser} -s "{subject}" --attach "{attachFile}" {recipient} < {textFile}'''
    cmd = cmd_template.format(
        emailUser = emailUser,
        recipient = recipient,
        subject   = subject,
        textFile  = fp.name,
        attachFile = attachFile
    )

    if attachFile == None:
        cmd = cmd.replace('--attach "None"', '')
    # end if

    process = subprocess.Popen(cmd, shell = True)
    process.wait()
# end def


In [8]:
emailUser = 'admin@i-kuanpern.com'
subject   = 'Bring weather equipment'

for subscriber in subscribers:
    text = 'Hi {username}, please bring {equipments} for the day'.format(
        username   = subscriber['username'],
        equipments = content
    )
    print text
    #utils_sendMail(emailUser, subscriber, subject, text, attachFile = None)
# end for

Hi admin, please bring umbrella, coat for the day
Hi kuanpern, please bring umbrella, coat for the day
